In [686]:
import pandas as pd
import numpy as np
import re

# 4번째(인덱스 3) + 5번째(인덱스 4) 행을 멀티헤더로 불러오기
df_raw = pd.read_excel("school_info_0306.xlsx", header=[3, 4])
# df_raw.columns

# 컬럼 공백, 개행 제거 및 멀티컬럼 평탄화
def clean_columns(df): 
    df.columns = [
        f"{a.strip().replace('\n', '').replace(' ', '')}_{b.strip().replace('\n', '').replace(' ', '')}"
        if 'Unnamed' not in str(b) else a.strip().replace('\n', '')
        for a, b in df.columns
    ]
    if df.columns[0].startswith("Unnamed"):
        df.drop(columns=[df.columns[0]], inplace=True)
    return df.reset_index(drop=True)
df_raw = clean_columns(df_raw)
df = df_raw.reset_index(drop=True)

# [전처리] 마지막 행 제거
# 이유: 엑셀 원본 마지막 행은 "합계" 같은 통계/요약 행으로, 실제 데이터가 아님
# df.shape[0]은 전체 행 수, df.index[-1]은 마지막 행 인덱스
df = df.drop(df.index[-1]).reset_index(drop=True)

In [687]:
# 컬럼 전처리
# [프로그램구분] - 개행 제거
df['프로그램구분'] = df['프로그램구분'].str.replace('\n', '', regex=False)

# [국가명] - 국가명 공백 제거
df['국가명'] = df['국가명'].str.strip()

# [전처리] 전체 데이터에서 \n 제거
# 문자열 컬럼에 대해서만 \n을 공백 없이 제거 (교환학생\n+인턴십 → 교환학생+인턴십)
# df = df.applymap(lambda x: x.replace('\n', '') if isinstance(x, str) else x)

# [대학명(국문)]
df['대학명(국문)'] = df['대학명(국문)'].str.replace('★', '', regex=False)
df['대학명(국문)'] = df['대학명(국문)'].str.replace('●', '', regex=False)
df['대학명(국문)'] = df['대학명(국문)'].str.strip()

# [대학명(영문)] - # 앞뒤 공백 제거
df['대학명(영문)'] = df['대학명(영문)'].str.strip()

# [유의사항] - ★, ● 제거
df['유의사항'] = df['유의사항'].str.replace('★ ', '', regex=False)

# [웹사이트] - 1) 2) 제거 후 ', '으로 대체
def clean_website_links(text):
    if pd.isna(text):
        return text
    # 링크만 추출
    links = re.findall(r'https?://[^\s\)]+', text)
    # 쉼표로 join 후 반환
    return ', '.join(links)
df['웹사이트'] = df['웹사이트'].apply(clean_website_links)

# [Factsheet 여부] - drop
df = df.drop(columns=['Factsheet 여부'])

# [교환학생수기 여부] - drop
df = df.drop(columns=['교환학생수기 여부'])
# df.columns

# [지원자격_최소학점]
def format_gpa(value):
    if pd.isna(value):
        return '-'

    value = str(value).replace('\n', '').strip()

    # 1. '3.0/4.5' 형태 → 그대로 + ' 이상'
    if re.fullmatch(r'\d+(\.\d+)?/4\.5', value):
        return f"{value} 이상"

    # 2. 'X.XX 이상' 형태 → 'X.XX/4.5 이상'
    match = re.fullmatch(r'(\d+(\.\d+)?) 이상', value)
    if match:
        return f"{match.group(1)}/4.5 이상"

    # 3. '학점 X.XX 이상' 형태 → 'X.XX/4.5 이상'
    match = re.fullmatch(r'학점\s*(\d+(\.\d+)?) 이상', value)
    if match:
        return f"{match.group(1)}/4.5 이상"

    # 4. 숫자만 있는 경우 (예: 2.81) → '2.81/4.5 이상'
    if re.fullmatch(r'\d+(\.\d+)?', value):
        return f"{value}/4.5 이상"

    # 5. 나머지 (문자 그대로 유지)
    return value

# [지원자격_어학성적] - , 제거 ("~점수 1,200이상"에서 ","로 인해 파싱 되는 경우 방지)
df['지원자격_어학성적'] = df['지원자격_어학성적'].str.replace(',', '', regex=False)

# 적용
df['지원자격_최소학점'] = df['지원자격_최소학점'].apply(format_gpa)
# df['지원자격_최소학점'].unique()

In [688]:
df[df['국가명'] == '일본'][['지역', '국가명', '대학명(국문)', '지원자격_어학성적']].apply(tuple, axis=1).unique()

array([('아시아', '일본', '삿포로가쿠인대(삿포로학원대)', 'C-2'),
       ('아시아', '일본', 'JF 오베린대', 'C-2'), ('아시아', '일본', '오사카경법대', 'C-2'),
       ('아시아', '일본', '신슈대', 'C-2'), ('아시아', '일본', '오츠마여대', 'C-2'),
       ('아시아', '일본', '나고야외국어대', ' C-2 A-2'),
       ('아시아', '일본', '칸다외국어대학', 'C-1 A-5'), ('아시아', '일본', '세츠난대학', 'C-2'),
       ('아시아', '일본', '일본경제대', 'C-2'), ('아시아', '일본', '사가여자단기대', 'C-2'),
       ('아시아', '일본', '국제기독교대', nan), ('아시아', '일본', '간세이가쿠인대', nan),
       ('아시아', '일본', '도시샤대', nan), ('아시아', '일본', 'JF 오베린대', nan),
       ('아시아', '일본', '오사카조가쿠인', nan), ('아시아', '일본', '히로시마조카쿠인대', nan),
       ('아시아', '일본', '코베대', nan), ('아시아', '일본', '세이난가쿠인대', nan),
       ('아시아', '일본', '난잔대', nan), ('아시아', '일본', '신슈대', 'A-2')],
      dtype=object)

In [689]:
print(f"전체 데이터 수 : {df.shape[0]}") # 컬럼행 제외, 실제 데이터 수
print("=" * 60)
for col in df.columns:
    print(f"[{col}]")
    print(f"고유값 수: {df[col].nunique()}")
    # print(f"고유값 리스트: {df[col].unique()[:10]}")  # 너무 많으면 10개만
    print(f"고유값 리스트: {df[col].unique()}") # 전체 표시
    print("-" * 60)

전체 데이터 수 : 338
[프로그램구분]
고유값 수: 5
고유값 리스트: ['일반교환' '방문교환' '인턴십 ' '특별' '교환학생+인턴십']
------------------------------------------------------------
[기관]
고유값 수: 4
고유값 리스트: ['자매대학' 'ACUCA' 'BCI' 'SAF']
------------------------------------------------------------
[일련번호]
고유값 수: 338
고유값 리스트: ['E0009' 'E0010' 'E0013' 'E0014' 'E0015' 'E0016' 'E0017' 'E0018' 'E0026'
 'E0027' 'E0031' 'E0034' 'E0036' 'E0037' 'E0038' 'E0039' 'E0040' 'E0041'
 'E0042' 'E0043' 'E0044' 'E0045' 'E0046' 'E0047' 'E0048' 'E0049' 'E0050'
 'E0051' 'E0052' 'E0054' 'E0055' 'E0057' 'E0058' 'E0060' 'E0061' 'E0062'
 'E0063' 'E0066' 'E0067' 'E0068' 'E0070' 'E0071' 'E0072' 'E0073' 'E0077'
 'E0079' 'E0080' 'E0081' 'E0082' 'E0083' 'E0084' 'E0085' 'E0086' 'E0087'
 'E0089' 'E0091' 'E0092' 'E0093' 'E0094' 'E0095' 'E0096' 'E0097' 'E0098'
 'E0099' 'E0100' 'E0101' 'E0102' 'E0103' 'E0104' 'E0105' 'E0106' 'E0107'
 'E0108' 'E0109' 'E0110' 'E0113' 'E0114' 'E0115' 'E0116' 'E0117' 'E0118'
 'E0119' 'E0120' 'E0122' 'E0123' 'E0124' 'E0126' 'E0127' 'E01

In [690]:
# [언어권구분]

COUNTRY_LANGUAGE_MAP = {
    # 아시아
    '일본': '일본어',
    '중국': '중국어',
    '대만': '중국어',

    # 유럽
    '프랑스': '프랑스어',
    '벨기에': '프랑스어',  # 네덜란드어도 일부 지역에서 사용되지만 기준표상 프랑스어 우선
    '스위스': '프랑스어',  # 복수 언어권 국가지만 프랑스어 우선
    '스페인': '프랑스어',   # DELE 자격 언급 기준으로 프랑스어로 간주
    '이탈리아': '프랑스어', # CELI 등 언급 기준으로 프랑스어(혹은 이탤리언 기준 적용시 따로 분기 가능)
    '독일': '독일어',
    '오스트리아': '독일어',
    '체코': '독일어',       # 일부 독일어 사용, 조건부 적용
    '루마니아': '프랑스어',
    '폴란드': '독일어',
    '헝가리': '독일어',
    '슬로바키아': '독일어',
    '핀란드': '영어(유럽)', # 자체 언어 있으나 영어 기준
    '덴마크': '영어(유럽)',
    '노르웨이': '영어(유럽)',
    '스웨덴': '영어(유럽)',
    '네덜란드': '영어(유럽)',
    '아일랜드': '영어(유럽)',
    '영국': '영어(유럽)',

    # 비유럽 영어권
    '미국': '영어(비유럽)',
    '캐나다': '영어(비유럽)',
    '호주': '영어(비유럽)',
    '뉴질랜드': '영어(비유럽)',
    '싱가포르': '영어(비유럽)',
    '말레이시아': '영어(비유럽)',

    # 중남미 및 기타
    '브라질': '기타',  # Celpe-Bras 등 포르투갈어 시험 존재
    '멕시코': '기타',
    '콜롬비아': '기타',
    '베트남': '기타',
    '러시아': '영어(유럽)',  # 영어 시험 기준 많음
    '튀르키예(터키)': '영어(유럽)',  # CEFR 기준 사용 많음

    # 기타국가 - 미지정시 '기타'로 처리
}

# 언어권별 키워드 배열 정의
LANGUAGE_KEYWORDS = {
    '영어': [
        'toefl', 'ibt', 'itp', 'ielts', 'toeic',
        'academic', 'duolingo', '듀오링고',
        '영어', '영어: toefl', '영어b2',
        'a-1', 'a-2', 'a-3', 'a-4', 'a-5',
        'a1', 'a2', 'a3', 'a4', 'a5'
    ],
    '프랑스어': [
        '프랑스어', 'delf', 'dele', 
        'delf b2', 'deleb1', 'delec1', 'delf 50/100',
        'd-1', 'd-2', 'd-3',
        'd1', 'd2', 'd3'
    ],
    '중국어': [
        '중국어', 'hsk', '신hsk',
        'b-3', 'b3',
        # 'b-1', 'b-2',  # 영어권에도 있는 등급
        # 'b1', 'b2',   # 영어권에도 있는 등급
    ],
    '일본어': [
        'jlpt', 'jpt', '일본어', 
        'n1', 'n2',
        'c-1', 'c-2',
        # 'c1', 'c2' # 독일어 CEFR 등급과 중복
    ],
    '독일어': [
        '독일어', 'zd','독일어e2',
        'e-1', 'e-2', 'e-3', 
        'e1', 'e2', 'e3', 
    ],
    '기타': [
        'celpe', '포르투갈어', 'intermediario', 'superior'
    ]
}

# 언어권 추출 함수
def extract_language_zones_with_country(row):
    raw_text = row['지원자격_어학성적']
    text = str(raw_text).lower().strip()
    country = str(row['국가명'])
    region = str(row['지역'])

    # 어학성적 비어 있거나 안내문이면 무조건 '기타'
    if pd.isna(raw_text) or text == '-' or '확인하시기 바랍니다' in text:
        return '기타'

    matched = set()

    for lang, keywords in LANGUAGE_KEYWORDS.items():
        if any(kw in text for kw in keywords):
            matched.add(lang)

    # b1~b2 계열 조건부 처리
    if re.search(r'\bb[- ]?[1-2]\b', text):
        if country in ['중국', '대만']:
            matched.add('중국어')
        elif region == '유럽':
            matched.add('영어')

    # 영어권 유럽/비유럽 세분화
    if '영어' in matched:
        matched.discard('영어')
        matched.add('영어(유럽)' if region == '유럽' else '영어(비유럽)')

    # 국가명 기반 언어권 보완
    default_lang = COUNTRY_LANGUAGE_MAP.get(country)
    if default_lang and default_lang not in matched:
        # 해당 언어권 키워드가 텍스트에 포함된 경우에만 추가
        if any(kw in text for kw in LANGUAGE_KEYWORDS.get(default_lang, [])):
            matched.add(default_lang)

    return ', '.join(sorted(matched)) if matched else '기타'
# 적용
df['언어권구분'] = df.apply(extract_language_zones_with_country, axis=1)

In [691]:
df[df['국가명']=='일본'][['지역', '국가명', '언어권구분', '지원자격_어학성적']].apply(tuple, axis=1).unique()

array([('아시아', '일본', '일본어', 'C-2'),
       ('아시아', '일본', '영어(비유럽), 일본어', ' C-2 A-2'),
       ('아시아', '일본', '영어(비유럽), 일본어', 'C-1 A-5'), ('아시아', '일본', '기타', nan),
       ('아시아', '일본', '영어(비유럽)', 'A-2')], dtype=object)

### 안되겠다 언어권구분까지만하고, explode는 gpt한테 맡겨야될듯

In [ ]:
df.to_csv('school_info_refined_0410.csv', index=False, encoding='utf-8-sig')

In [674]:
import pandas as pd

# 출력 행 수를 무제한으로 설정
pd.set_option('display.max_rows', None)

# 콤마 포함된 '언어권구분'을 가진 행만 필터링하여 출력
filtered_df = df[df['언어권구분'].str.contains(',', na=False)][['지역', '국가명', '언어권구분', '지원자격_어학성적']]
print(filtered_df)


      지역      국가명          언어권구분  \
9     유럽       독일    독일어, 영어(유럽)   
10    유럽       독일    독일어, 영어(유럽)   
11    유럽       독일    독일어, 영어(유럽)   
12    유럽       독일    독일어, 영어(유럽)   
13    유럽       독일    독일어, 영어(유럽)   
25    유럽      벨기에   영어(유럽), 프랑스어   
29    유럽      스페인   영어(유럽), 프랑스어   
30    유럽      스페인   영어(유럽), 프랑스어   
31    유럽      스페인   영어(유럽), 프랑스어   
32    유럽      스페인   영어(유럽), 프랑스어   
33    유럽      스페인   영어(유럽), 프랑스어   
34    유럽      스페인   영어(유럽), 프랑스어   
35    유럽      스페인   영어(유럽), 프랑스어   
36    유럽      스페인   영어(유럽), 프랑스어   
37    유럽      스페인   영어(유럽), 프랑스어   
38    유럽      스페인   영어(유럽), 프랑스어   
39    유럽      스페인   영어(유럽), 프랑스어   
76    유럽      프랑스   영어(유럽), 프랑스어   
77    유럽      프랑스   영어(유럽), 프랑스어   
79    유럽      프랑스   영어(유럽), 프랑스어   
80    유럽      프랑스   영어(유럽), 프랑스어   
82    유럽      프랑스   영어(유럽), 프랑스어   
83    유럽      프랑스   영어(유럽), 프랑스어   
86    유럽      프랑스   영어(유럽), 프랑스어   
87    유럽      프랑스   영어(유럽), 프랑스어   
89    유럽      프랑스   영어(유럽), 프랑스어   
90    유럽      프랑스   영어(유럽), 

In [684]:
import pandas as pd
import numpy as np
import re

# 1. 리스트로 분리 (콤마 및 개행 처리)
df['언어권_list'] = df['언어권구분'].fillna('').apply(lambda x: [i.strip() for i in x.split(',')])
df['어학성적_list'] = df['지원자격_어학성적'].fillna('').apply(lambda x: [i.strip() for i in re.split(r',|\n', x)])

# 2. 리스트 길이 맞추기
def pad_lists(row):
    len_lang = len(row['언어권_list'])
    len_score = len(row['어학성적_list'])
    max_len = max(len_lang, len_score)
    lang_list = row['언어권_list'] + [np.nan] * (max_len - len_lang)
    score_list = row['어학성적_list'] + [np.nan] * (max_len - len_score)
    return pd.Series({'언어권_list': lang_list, '어학성적_list': score_list})

df[['언어권_list', '어학성적_list']] = df.apply(pad_lists, axis=1)

# 3. explode 전 zip
df['언어권_어학성적'] = df.apply(lambda row: list(zip(row['언어권_list'], row['어학성적_list'])), axis=1)
df_exploded = df.explode('언어권_어학성적')

# 4. 언어권, 어학성적 분리
df_exploded[['언어권', '어학성적']] = pd.DataFrame(df_exploded['언어권_어학성적'].tolist(), index=df_exploded.index)

# 5. NaN 언어권 자동 보정 로직
def infer_language_zone(row):
    score = str(row['어학성적']).lower()
    country = row['국가명']
    region = row['지역']

    if pd.notna(row['언어권']):
        return row['언어권']

    # 영어 시험
    if any(k in score for k in ['toefl', 'ielts', 'toeic', 'english', 'b2', 'a-2', 'a-3']):
        if region == '유럽':
            return '영어(유럽)'
        else:
            return '영어(비유럽)'
    
    # 독일어
    if '독일어' in score or re.search(r'\bE[1-5]\b', score):
        return '독일어'
    
    # 프랑스어
    if 'francais' in score or 'delf' in score or re.search(r'\bD-[1-5]\b', score):
        return '프랑스어'
    
    # 일본어
    if 'jlpt' in score or 'jpt' in score or re.search(r'\bC-[1-2]\b', score):
        if country == '일본':
            return '일본어'

    # 중국어
    if 'hsk' in score or re.search(r'\bB-[1-6]\b', score):
        if country == '중국':
            return '중국어'

    return np.nan

# 6. 적용
df_exploded['언어권'] = df_exploded.apply(infer_language_zone, axis=1)

# 7. 필요한 컬럼만 출력
result = df_exploded[['지역', '대학명(국문)', '국가명', '언어권', '어학성적']]


In [685]:
# df[df['국가명']=='일본'][['지역', '국가명', '언어권구분', '지원자격_어학성적']].apply(tuple, axis=1).unique()
# df_exploded[['지역', '대학명(국문)', '언어권구분', '지원자격_어학성적','국가명', '언어권', '어학성적']]
df_exploded[
    df_exploded['언어권구분'].str.contains(',', na=False)
][['지역', '대학명(국문)', '언어권구분', '지원자격_어학성적', '국가명', '언어권', '어학성적']]


,지역,대학명(국문),언어권구분,지원자격_어학성적,국가명,언어권,어학성적
9,유럽,카이저슬라우턴대,"독일어, 영어(유럽)",TOEFL(iBT) 80\nIELTS 6.0\nTOEIC 850\n독일어 E2,독일,독일어,TOEFL(iBT) 80
9,유럽,카이저슬라우턴대,"독일어, 영어(유럽)",TOEFL(iBT) 80\nIELTS 6.0\nTOEIC 850\n독일어 E2,독일,영어(유럽),IELTS 6.0
9,유럽,카이저슬라우턴대,"독일어, 영어(유럽)",TOEFL(iBT) 80\nIELTS 6.0\nTOEIC 850\n독일어 E2,독일,영어(유럽),TOEIC 850
9,유럽,카이저슬라우턴대,"독일어, 영어(유럽)",TOEFL(iBT) 80\nIELTS 6.0\nTOEIC 850\n독일어 E2,독일,독일어,독일어 E2
10,유럽,보름스대,"독일어, 영어(유럽)",영어B2 \n독일어E2,독일,독일어,영어B2
10,유럽,보름스대,"독일어, 영어(유럽)",영어B2 \n독일어E2,독일,영어(유럽),독일어E2
11,유럽,로이파나대,"독일어, 영어(유럽)",영어B2 \n독일어E2,독일,독일어,영어B2
11,유럽,로이파나대,"독일어, 영어(유럽)",영어B2 \n독일어E2,독일,영어(유럽),독일어E2
12,유럽,플렌스부르크대,"독일어, 영어(유럽)",TOEFL(iBT) 78\n독일어 E1,독일,독일어,TOEFL(iBT) 78
12,유럽,플렌스부르크대,"독일어, 영어(유럽)",TOEFL(iBT) 78\n독일어 E1,독일,영어(유럽),독일어 E1


In [644]:
df[['언어권구분', '지원자격_어학성적']].apply(tuple, axis=1).unique()

array([('프랑스어', 'D-1'), ('영어(유럽)', 'IELTS 6.0\nTOEFL (iBT) 80'),
       ('영어(유럽)', 'TOEFL(iBT) 80\nIELTS 6.0\nTOEIC 960'),
       ('영어(유럽)', 'TOEFL iBT 80\nIELTS 6.0\nTOEIC 1,200 이상'),
       ('영어(유럽)', '영어 B2'), ('영어(유럽)', 'A-2'),
       ('영어(유럽)', 'TOEFL(iBT) 83\nIELTS 6.5'),
       ('독일어, 영어(유럽)', 'TOEFL(iBT) 80\nIELTS 6.0\nTOEIC 850\n독일어 E2'),
       ('독일어, 영어(유럽)', '영어B2 \n독일어E2'),
       ('독일어, 영어(유럽)', 'TOEFL(iBT) 78\n독일어 E1'),
       ('독일어, 영어(유럽)', '영어 B2\n독일어 E2'), ('영어(유럽)', 'B2'),
       ('영어(유럽)', 'TOEFL(iBT) 75\nIELTS 5.5\nTOEIC 800'),
       ('영어(유럽)', 'TOEFL 80\nIELTS 6.0'),
       ('영어(유럽)', 'TOEFL(iBT) 80\nIELTS 6.0\nTOEIC 850'),
       ('영어(유럽), 프랑스어', '영어 B2 \n프랑스어 D2'),
       ('영어(유럽)', 'TOEFL(iBT) 78\nIELTS 6.0'),
       ('영어(유럽)', 'TOEFL(iBT) 75\nIELTS 5.5'),
       ('영어(유럽), 프랑스어', '영어 B2\n DELE B1'),
       ('영어(유럽), 프랑스어', 'TOEFL(iBT) 75\nIELTS 5.5\nTOEIC 800\n DELE B1'),
       ('영어(유럽), 프랑스어', '영어 B2 \nDELE B1'),
       ('영어(유럽), 프랑스어', '영어 B2\nDELE B1'),
 

In [639]:
import pandas as pd
import re

# ------------------------------
# 시험-언어권 기준표
# ------------------------------
exam_lang_map = {
    'TOEFL': '영어',
    'IELTS': '영어',
    'TOEIC': '영어',
    'TOEFL ITP': '영어',
    'DELF': '프랑스어',
    'ZD': '독일어',
    'HSK': '중국어',
    'JLPT': '일본어',
    'JPT': '일본어'
}

# ------------------------------
# 등급 기준표 정의
# ------------------------------
english_non_eu = {
    'TOEFL': {85: 'A1', 80: 'A2', 75: 'A3', 70: 'A4', 60: 'A5'},
    'IELTS': {6.5: 'A1', 6.0: 'A2', 5.5: 'A3', 5.0: 'A4', 4.5: 'A5'},
    'TOEIC': {900: 'A1', 850: 'A2', 800: 'A3', 750: 'A4', 700: 'A5'},
    'TOEFL ITP': {600: 'A1', 560: 'A2', 545: 'A3', 530: 'A4', 515: 'A5'},
}
english_eu = {
    'TOEFL': {114: 'C2', 95: 'C1', 72: 'B2', 42: 'B1'},
    'IELTS': {9.0: 'C2', 7.0: 'C1', 5.5: 'B2', 4.0: 'B1'},
    'TOEIC': {945: 'C1', 785: 'B2', 550: 'B1', 225: 'A2'},
}
french = {'DELF': {'B2': 'D1', 'B1': 'D2', 'A2': 'D3'}}
german = {'ZD': {'B2': 'E1', 'B1': 'E2', 'A2': 'E3'}}
chinese = {'HSK': {'6급': 'B1', '5급': 'B2', '4급': 'B3'}}
japanese = {'JLPT': {'N1': 'C1', 'N2': 'C2'}, 'JPT': {900: 'C1', 600: 'C2'}}

# ------------------------------
# 점수 추출 및 변환 함수
# ------------------------------
def extract_scores(text):
    patterns = {
        'TOEFL': r'TOEFL(?:\(iBT\)| iBT)?[^\d]*(\d{2,3})',
        'IELTS': r'IELTS[^\d]*(\d(?:\.\d)?)',
        'TOEIC': r'TOEIC[^\d]*(\d{3,4})',
        'TOEFL ITP': r'TOEFL ITP[^\d]*(\d{3})',
        'DELF': r'DELF[^\w]*(B2|B1|A2)',
        'ZD': r'독일어[^\w]*(B2|B1|A2)',
        'HSK': r'HSK[^\d]*(\d)[^\급]*급',
        'JLPT': r'JLPT[^\w]*(N1|N2)',
        'JPT': r'JPT[^\d]*(\d{3})'
    }
    results = []
    for test, pattern in patterns.items():
        matches = re.findall(pattern, text if pd.notnull(text) else "", flags=re.IGNORECASE)
        for m in matches:
            results.append((test, m.strip()))
    return results

def map_score_to_grade(score, mapping, reverse=True):
    try:
        s = float(score)
        sorted_keys = sorted(mapping.keys(), reverse=reverse)
        for key in sorted_keys:
            if s >= key:
                return mapping[key]
    except:
        return None

def get_english_grade(score, test_name, is_europe=True):
    mapping = english_eu if is_europe else english_non_eu
    table = mapping.get(test_name)
    if table:
        return map_score_to_grade(score, table)
    return None

def determine_region(lang_type):
    if '영어(유럽)' in lang_type:
        return '유럽'
    elif '영어(비유럽)' in lang_type:
        return '비유럽'
    return None

def get_grade(test, score, region):
    if test in ['TOEFL', 'IELTS', 'TOEIC', 'TOEFL ITP']:
        return get_english_grade(score, test, is_europe=(region == '유럽'))
    elif test == 'DELF':
        return french['DELF'].get(score)
    elif test == 'ZD':
        return german['ZD'].get(score)
    elif test == 'HSK':
        return chinese['HSK'].get(f"{score}급")
    elif test == 'JLPT':
        return japanese['JLPT'].get(score)
    elif test == 'JPT':
        return japanese['JPT'].get(int(score))
    return None

# ------------------------------
# 점수 기반 df_lang 생성
# ------------------------------
rows = []
for _, row in df.iterrows():
    text = row['지원자격_어학성적']
    lang_type = row['언어권구분']
    country = row['국가명']
    region = determine_region(lang_type)
    
    for test, score in extract_scores(text):
        grade = get_grade(test, score, region)
        lang_inferred = exam_lang_map.get(test)
        if lang_inferred and lang_inferred in lang_type:
            rows.append({
                '일련번호': row['일련번호'],
                '국가명': country,
                '언어권구분': lang_inferred,
                '지원자격_어학성적': text,
                '어학시험명': test,
                '어학시험점수': score,
                '숭실대공인어학성적': grade
            })

df_lang = pd.DataFrame(rows)

# ------------------------------
# 등급 기반 explode 처리
# ------------------------------
grade_pattern = r'(A-[1-5]|B-[1-3]|C-[1-2]|D-[1-3]|E-[1-3])'
grade_order = [
    'A-1', 'A-2', 'A-3', 'A-4', 'A-5',
    'B-1', 'B-2', 'B-3',
    'C-1', 'C-2',
    'D-1', 'D-2', 'D-3',
    'E-1', 'E-2', 'E-3'
]
grade_rank_dict = {grade: i for i, grade in enumerate(grade_order)}

def extract_grades(text):
    if pd.isna(text):
        return []
    return re.findall(grade_pattern, text)

def split_langs(text):
    if pd.isna(text):
        return []
    return [s.strip() for s in str(text).split(',')]

# 기존 '언어권구분' 컬럼 제거 (중복 방지)
df_base = df[['일련번호', '국가명', '지원자격_어학성적']].copy()
df_base['등급목록'] = df['지원자격_어학성적'].apply(extract_grades)
df_base['언어권목록'] = df['언어권구분'].apply(split_langs)

# 등급 먼저 explode하고 인덱스 초기화 후, 언어권 explode
df_exploded = df_base.explode('등급목록').dropna(subset=['등급목록']).reset_index(drop=True)
df_exploded = df_exploded.explode('언어권목록').dropna(subset=['언어권목록']).reset_index(drop=True)

# 컬럼 이름 정리 (중복 방지됨)
df_exploded = df_exploded.rename(columns={
    '등급목록': '추출된등급',
    '언어권목록': '언어권구분'
})


# 등급-언어권 필터링
df_exploded = df_exploded[
    ((df_exploded['추출된등급'].str.startswith('C-')) & (df_exploded['국가명'] == '일본') & (df_exploded['언어권구분'] == '일본어')) |
    ((df_exploded['추출된등급'].str.startswith('B-')) & (df_exploded['국가명'] == '중국') & (df_exploded['언어권구분'] == '중국어')) |
    ((df_exploded['추출된등급'].str.startswith('D-')) & (df_exploded['언어권구분'] == '프랑스어')) |
    ((df_exploded['추출된등급'].str.startswith('E-')) & (df_exploded['언어권구분'] == '독일어')) |
    ((df_exploded['추출된등급'].str.startswith('A-')) & df_exploded['언어권구분'].str.contains('영어')) |
    ((df_exploded['추출된등급'].str.startswith('B-')) & df_exploded['언어권구분'].str.contains('영어')) |
    ((df_exploded['추출된등급'].str.startswith('C-')) & df_exploded['언어권구분'].str.contains('영어'))
]


# 등급 정보 정리
df_exploded['어학시험명'] = None
df_exploded['어학시험점수'] = None
df_exploded['숭실대공인어학성적'] = df_exploded['추출된등급']
df_exploded['grade_rank'] = df_exploded['숭실대공인어학성적'].map(grade_rank_dict)
df_exploded['숭실대공인어학성적기준(최소)'] = df_exploded['숭실대공인어학성적']

# ------------------------------
# 병합
# ------------------------------
existing_ids = df_lang['일련번호'].unique()
df_exploded_filtered = df_exploded[~df_exploded['일련번호'].isin(existing_ids)]

df_lang_final = pd.concat([df_lang, df_exploded_filtered], ignore_index=True)
cols_to_merge = ['일련번호', '언어권구분', '어학시험명', '어학시험점수', '숭실대공인어학성적', 'grade_rank', '숭실대공인어학성적기준(최소)']
df_final = pd.merge(df, df_lang_final[cols_to_merge], on='일련번호', how='left')


In [640]:
df_final.columns

Index(['프로그램구분', '기관', '일련번호', '지역', '국가명', '대학명(국문)', '대학명(영문)', '지원자격_최소학점',
       '지원자격_어학성적', '지원자격_특이사항', '참고사항', '수학가능학과/영어강의목록 등', '유의사항', '웹사이트',
       '언어권구분_x', '언어권구분_y', '어학시험명', '어학시험점수', '숭실대공인어학성적', 'grade_rank',
       '숭실대공인어학성적기준(최소)'],
      dtype='object')

In [643]:
df_final[df_final['국가명']=='일본'][['대학명(국문)', '국가명', '언어권구분_x', '언어권구분_y', '지원자격_어학성적', '어학시험명', '어학시험점수', '숭실대공인어학성적', 'grade_rank', '숭실대공인어학성적기준(최소)']].apply(tuple, axis=1).unique()

array([('삿포로가쿠인대(삿포로학원대)', '일본', '일본어', '일본어', 'C-2', None, None, 'C-2', 9.0, 'C-2'),
       ('JF 오베린대', '일본', '일본어', '일본어', 'C-2', None, None, 'C-2', 9.0, 'C-2'),
       ('오사카경법대', '일본', '일본어', '일본어', 'C-2', None, None, 'C-2', 9.0, 'C-2'),
       ('신슈대', '일본', '일본어', '일본어', 'C-2', None, None, 'C-2', 9.0, 'C-2'),
       ('오츠마여대', '일본', '일본어', '일본어', 'C-2', None, None, 'C-2', 9.0, 'C-2'),
       ('나고야외국어대', '일본', '영어(비유럽), 일본어', '영어(비유럽)', ' C-2, A-2', None, None, 'C-2', 9.0, 'C-2'),
       ('나고야외국어대', '일본', '영어(비유럽), 일본어', '일본어', ' C-2, A-2', None, None, 'C-2', 9.0, 'C-2'),
       ('나고야외국어대', '일본', '영어(비유럽), 일본어', '영어(비유럽)', ' C-2, A-2', None, None, 'A-2', 1.0, 'A-2'),
       ('칸다외국어대학', '일본', '영어(비유럽), 일본어', '영어(비유럽)', 'C-1, A-5', None, None, 'C-1', 8.0, 'C-1'),
       ('칸다외국어대학', '일본', '영어(비유럽), 일본어', '일본어', 'C-1, A-5', None, None, 'C-1', 8.0, 'C-1'),
       ('칸다외국어대학', '일본', '영어(비유럽), 일본어', '영어(비유럽)', 'C-1, A-5', None, None, 'A-5', 4.0, 'A-5'),
       ('세츠난대학', '일본', '일본어', '일본어', '

In [627]:
df_final[df_final['국가명']=='프랑스'][['대학명(국문)', '국가명', '언어권구분', '지원자격_어학성적', '어학시험명', '어학시험점수', '숭실대공인어학성적', 'grade_rank', '숭실대공인어학성적기준(최소)']].apply(tuple, axis=1).unique()

KeyError: "['언어권구분'] not in index"